PyTorch Regression Example with California Housing Dataset
==========================================================

This script demonstrates how to build, train, and evaluate a simple deep learning regression model using PyTorch. The California Housing dataset is used for predicting house prices based on various features.

In [ ]:
# !pip install torch torchvision torchaudio --quiet
# !pip install scikit-learn --quiet
# !pip install matplotlib --quiet
# !pip install torchviz --quiet


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
!nvidia-smi

Sat Nov  9 15:29:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1. Load and preprocess the data

In [ ]:
# Load the California Housing dataset
california = fetch_california_housing()

In [ ]:
# The data is in california.data, target is in california.target
X = california.data
y = california.target

# Convert to pandas DataFrame for easier manipulation (optional)
feature_names = california.feature_names
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

In [ ]:
# Split the data into training, validation, and test sets
X_trainval, X_test, y_trainval, y_test = train_test_split(
    df.iloc[:, :-1], df['target'], test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [ ]:
# Feature Scaling
# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# 2. Create custom Dataset class

In [ ]:
class CaliforniaHousingDataset(Dataset):
    def __init__(self, features, targets):
        """
        Args:
            features (numpy.ndarray): Features data.
            targets (numpy.ndarray): Target data.
        """
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32).unsqueeze(1)  # Make it Nx1

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the data point.
        Returns:
            (tuple): (feature, target) of the given index.
        """
        return self.features[idx], self.targets[idx]

In [ ]:
# Create Dataset objects
train_dataset = CaliforniaHousingDataset(X_train, y_train.values)
val_dataset = CaliforniaHousingDataset(X_val, y_val.values)
test_dataset = CaliforniaHousingDataset(X_test, y_test.values)

# 3. Create DataLoaders

In [ ]:
batch_size = 64

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

# 4. Define the Neural Network Model

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        """
        Args:
            input_size (int): Number of input features.
        """
        super(RegressionModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 64),  # First hidden layer with 64 neurons
            nn.ReLU(),
            nn.Linear(64, 32),          # Second hidden layer with 32 neurons
            nn.ReLU(),
            nn.Linear(32, 1)            # Output layer
        )

    def forward(self, x):
        """
        Forward pass of the neural network.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.network(x)

In [ ]:
# Instantiate the model, define loss function and optimizer
input_size = X_train.shape[1]
model = RegressionModel(input_size).to(device)

In [ ]:
print(model)

RegressionModel(
  (network): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [ ]:
from torchsummary import summary

# Get the model summary
summary(model, input_size=(input_size,), device=str(device))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             576
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 32]           2,080
              ReLU-4                   [-1, 32]               0
            Linear-5                    [-1, 1]              33
Total params: 2,689
Trainable params: 2,689
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [ ]:
from torchinfo import summary

# Get the detailed model summary
summary(model, input_size=(batch_size, input_size), device=str(device))


Layer (type:depth-idx)                   Output Shape              Param #
RegressionModel                          [64, 1]                   --
├─Sequential: 1-1                        [64, 1]                   --
│    └─Linear: 2-1                       [64, 64]                  576
│    └─ReLU: 2-2                         [64, 64]                  --
│    └─Linear: 2-3                       [64, 32]                  2,080
│    └─ReLU: 2-4                         [64, 32]                  --
│    └─Linear: 2-5                       [64, 1]                   33
Total params: 2,689
Trainable params: 2,689
Non-trainable params: 0
Total mult-adds (M): 0.17
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.01
Estimated Total Size (MB): 0.06

This will output a detailed summary, including:

- Layer hierarchy
- Output shapes at each layer
- Number of trainable and non-trainable parameters
- Total parameters and memory usage


In [ ]:
# Define a dummy input tensor with the correct input size
dummy_input = torch.randn(1, input_size).to(device)

# Export the model to an ONNX file
torch.onnx.export(
    model,               # Your PyTorch model
    dummy_input,         # An example input tensor
    "churn_model.onnx",  # The file name to save the ONNX model
    input_names=['input'],   # Name of the input node
    output_names=['output'], # Name of the output node
    opset_version=11         # ONNX version
)


In [ ]:
# Define loss function (Mean Squared Error for regression)
criterion = nn.MSELoss()

# Define optimizer (Adam optimizer)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 5. Training Loop

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    running_loss = 0.0

    for features, targets in train_loader:
        # Move data to the device
        features = features.to(device)
        targets = targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(features)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item() * features.size(0)

    # Calculate average loss over the epoch
    epoch_loss = running_loss / len(train_loader.dataset)

    # Evaluate on validation set
    model.eval()
    val_running_loss = 0.0

    with torch.no_grad():
        for features, targets in val_loader:
            features = features.to(device)
            targets = targets.to(device)

            outputs = model(features)
            loss = criterion(outputs, targets)
            val_running_loss += loss.item() * features.size(0)

    val_loss = val_running_loss / len(val_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}")

Epoch [1/100], Training Loss: 1.4710, Validation Loss: 1.0558
Epoch [2/100], Training Loss: 0.5217, Validation Loss: 0.4435
Epoch [3/100], Training Loss: 0.4150, Validation Loss: 0.4255
Epoch [4/100], Training Loss: 0.3936, Validation Loss: 0.4101
Epoch [5/100], Training Loss: 0.3816, Validation Loss: 0.3927
Epoch [6/100], Training Loss: 0.3728, Validation Loss: 0.3776
Epoch [7/100], Training Loss: 0.3677, Validation Loss: 0.3746
Epoch [8/100], Training Loss: 0.3613, Validation Loss: 0.3824
Epoch [9/100], Training Loss: 0.3517, Validation Loss: 0.3704
Epoch [10/100], Training Loss: 0.3450, Validation Loss: 0.3840
Epoch [11/100], Training Loss: 0.3432, Validation Loss: 0.3565
Epoch [12/100], Training Loss: 0.3351, Validation Loss: 0.3689
Epoch [13/100], Training Loss: 0.3293, Validation Loss: 0.3709
Epoch [14/100], Training Loss: 0.3275, Validation Loss: 0.3675
Epoch [15/100], Training Loss: 0.3268, Validation Loss: 0.4201
Epoch [16/100], Training Loss: 0.3256, Validation Loss: 0.3823
E

# 6. Evaluation on Test Set

In [ ]:
# Set model to evaluation mode
model.eval()

test_running_loss = 0.0

with torch.no_grad():
    for features, targets in test_loader:
        features = features.to(device)
        targets = targets.to(device)

        outputs = model(features)
        loss = criterion(outputs, targets)
        test_running_loss += loss.item() * features.size(0)

test_loss = test_running_loss / len(test_loader.dataset)

print(f"Test Loss: {test_loss:.4f}")

Test Loss: 0.2882


In [ ]:
# Optionally, compute additional metrics like R-squared
from sklearn.metrics import r2_score

In [ ]:
# Collect all predictions and true values
all_preds = []
all_targets = []

with torch.no_grad():
    for features, targets in test_loader:
        features = features.to(device)
        targets = targets.to(device)

        outputs = model(features)

        all_preds.append(outputs.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

all_preds = np.concatenate(all_preds, axis=0)
all_targets = np.concatenate(all_targets, axis=0)

In [ ]:
# Compute R-squared
r2 = r2_score(all_targets, all_preds)

print(f"R-squared on Test Set: {r2:.4f}")

R-squared on Test Set: 0.7801
